In [23]:
# 평가 지표 평균 90 이상
import pandas as pd
from numpy import mean, std
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from imblearn.under_sampling import TomekLinks
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from imblearn.under_sampling import *

df = pd.read_csv('./TravelInsurancePrediction.csv')
df['FrequentFlyer'] = df['FrequentFlyer'].map({'Yes': 1, 'No': 0})
df['EverTravelledAbroad'] = df['EverTravelledAbroad'].map({'Yes': 1, 'No': 0})
df["Employment Type"] = df["Employment Type"].map({"Government Sector" : 1, "Private Sector/Self Employed" : 0})
df['GraduateOrNot'] = df['GraduateOrNot'].map({'Yes': 1, 'No': 0})

# df = df.drop(["GraduateOrNot"] , axis=1)
df_index = ['Age', 'AnnualIncome', 'ChronicDiseases',"GraduateOrNot", 
          'FrequentFlyer', 'FamilyMembers', 'Employment Type',
          'EverTravelledAbroad']
df_label_index = ['TravelInsurance']
ndf = df[['Age', 'AnnualIncome', 'ChronicDiseases', 'GraduateOrNot',
          'FrequentFlyer', 'FamilyMembers', 'Employment Type',
          'EverTravelledAbroad', 'TravelInsurance']]
X = ndf[list(ndf.columns)[:-1]]
y = ndf[list(ndf.columns)[-1]]
# y = y.astype({'TravelInsurance': int})
# X = preprocessing.StandardScaler().fit(X).transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=10)
print(X_train.shape)
df_new = pd.concat([X_train,y_train], axis = 1)

df_new.head()

(1589, 8)


,Age,AnnualIncome,ChronicDiseases,GraduateOrNot,FrequentFlyer,FamilyMembers,Employment Type,EverTravelledAbroad,TravelInsurance
1857,27,900000,0,1,0,4,1,0,0
685,34,1500000,1,1,1,4,0,1,1
1970,29,500000,0,1,0,4,0,0,1
1885,35,1100000,0,1,0,3,1,0,0
895,28,600000,0,1,0,5,0,1,1


In [ ]:
# incom>132500 이상, 가족이 5.5 명 이상 이걸 오버샘플링해보자
ndf1 = df[['Age', 'AnnualIncome', 'Employment Type',
           'TravelInsurance']]
X1 = ndf[list(ndf.columns)[:-1]]
y1 = ndf[list(ndf.columns)[-1]]
# y = y.astype({'TravelInsurance': int})
# X = preprocessing.StandardScaler().fit(X).transform(X)

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, 
                                                    random_state=10)
df_new = pd.concat([X_train,y_train], axis = 1)

df_new = df_new[(df_new['AnnualIncome'] >= 1275000) | (df_new['FamilyMembers'] >= 5.5) | (df_new['Age'] >= 32.5)| (df_new['GraduateOrNot'] >= 0.5) ]
X_train = df_new[list(ndf.columns)[:-1]]

y_train = df_new[list(ndf.columns)[-1]]
df_new.info()
X_train.info()

In [ ]:
df_new.describe()

#### 언더샘플링

In [ ]:
# 평균 73, 재현율 70( 1) 그러나 (0 )이 70 대로 떨어짐
enn = EditedNearestNeighbours()
X_resampled, y_resampled  = enn.fit_resample(X_train, y_train)
print(X_resampled.shape)

In [ ]:
# 재현율 57 
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
print(X_resampled.shape)

In [ ]:
from imblearn.under_sampling import NearMiss
nm1 = NearMiss(version=1)
X_resampled, y_resampled = nm1.fit_resample(X_train, y_train)
print(X_resampled.shape)

In [ ]:
renn = RepeatedEditedNearestNeighbours()
X_resampled, y_resampled = renn.fit_resample(X_train, y_train)
print(X_resampled.shape)

In [ ]:
allknn = AllKNN()
X_resampled, y_resampled = allknn.fit_resample(X_train, y_train)
print(X_resampled.shape)

In [ ]:
#언더 샘플링 중에는 제일 좋네 그나마
cnn = CondensedNearestNeighbour(random_state=0)
X_resampled, y_resampled = cnn.fit_resample(X_train, y_train)
print(X_resampled.shape)

In [ ]:
# 니가 일등이네
oss = OneSidedSelection(random_state=0)
X_resampled, y_resampled = oss.fit_resample(X_train, y_train)
print(X_resampled.shape)

In [ ]:
ncr = NeighbourhoodCleaningRule()
X_resampled, y_resampled = ncr.fit_resample(X_train, y_train)
print(X_resampled.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
iht = InstanceHardnessThreshold(random_state=0,                                 estimator=LogisticRegression(
                                    solver='lbfgs', multi_class='auto'))
X_resampled, y_resampled = iht.fit_resample(X_train, y_train)
print(X_resampled.shape)

#### 오버샘플링 smotenc / smoten

In [10]:
# 가장 높은 점수 결정 나무, depth 5
from imblearn.over_sampling import SMOTENC
smote_nc = SMOTENC(categorical_features=[0, 2], random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train)
print(X_resampled.shape)

(2040, 13)


In [ ]:
from imblearn.over_sampling import SMOTEN
sampler = SMOTEN(random_state=0)
X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
print(X_resampled.shape)

#### 오버와 언더 조합

In [ ]:
from imblearn.combine import *
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
print(X_resampled.shape)

In [ ]:
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)
print(X_resampled.shape)

#### 분류 모델

In [11]:
from sklearn import tree
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

tree_model.fit(X_resampled, y_resampled)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)


[[250   7]
 [ 58  83]]
              precision    recall  f1-score   support

           0       0.81      0.97      0.88       257
           1       0.92      0.59      0.72       141

    accuracy                           0.84       398
   macro avg       0.87      0.78      0.80       398
weighted avg       0.85      0.84      0.83       398



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)
bbc.fit(X_resampled, y_resampled) 
y_hat = bbc.predict(X_test)
bbc_matrix = metrics.confusion_matrix(y_test, y_hat)
print(bbc_matrix)
bbc_report = metrics.classification_report(y_test, y_hat)
print(bbc_report)

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brf.fit(X_resampled, y_resampled) 
y_hat = brf.predict(X_test)
brf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(brf_matrix)
brf_report = metrics.classification_report(y_test, y_hat)
print(brf_report)

In [ ]:
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=0)
eec.fit(X_resampled, y_resampled)
y_hat = eec.predict(X_test)
eec_matrix = metrics.confusion_matrix(y_test, y_hat)
print(eec_matrix)
eec_report = metrics.classification_report(y_test, y_hat)
print(eec_report)

#### 랜덤포레스트

In [ ]:
#랜덤 포레스트는 오버샘플링이 좋음
# 오버 언더 조합도 낫베드( 데이터 개수가 늘어날 수록 좋음)
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_resampled , y_resampled)
# rf_clf.fit(X_train , y_train)
y_hat = rf_clf.predict(X_test)
rf_clf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_clf_matrix)
rf_clf_report = metrics.classification_report(y_test, y_hat)
print(rf_clf_report)

#### 결정트리 시각화

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(tree_model, out_file="tree.dot", 
                class_names=df_index , 
                feature_names = df_index, 
                impurity=True, 
                filled=True)

In [ ]:
import graphviz

# 위에서 생성된 tree.dot 파일을 Graphviz 읽어서 Jupyter Notebook상에서 시각화 
with open("tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

#### GBM

In [ ]:
from lightgbm import LGBMClassifier
lgbm_wrapper = LGBMClassifier(n_estimators=400)

# LightGBM도 XGBoost와 동일하게 조기 중단 수행 가능. 
evals = [(X_test, y_test)]
lgbm_wrapper.fit(X_resampled, y_resampled, early_stopping_rounds=100, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
y_hat = lgbm_wrapper.predict(X_test)
lgbm_wrapper_matrix = metrics.confusion_matrix(y_test, y_hat)
print(lgbm_wrapper_matrix)
lgbm_wrapper_report = metrics.classification_report(y_test, y_hat)
print(lgbm_wrapper_report)

#### recall / precision

- recall 은 대상물체를 빠뜨리지 않고 얼마나 잘 잡아내느냐
- precision은 검출된 결과가 얼마나 정확한가

#### xgboost

In [ ]:
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

dtrain = xgb.DMatrix(data=X_train , label=y_train)
dtest = xgb.DMatrix(data=X_test , label=y_test)
params = { 'max_depth':3,
           'eta': 0.1,
           'objective':'binary:logistic',
           'eval_metric':'logloss',
           'early_stoppings':100
        }
num_rounds = 400
# train 데이터 셋은 ‘train’ , evaluation(test) 데이터 셋은 ‘eval’ 로 명기합니다. 
wlist = [(dtrain,'train'),(dtest,'eval') ]
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params , dtrain=dtrain , num_boost_round=num_rounds , evals=wlist )
pred_probs = xgb_model.predict(dtest)

preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

get_clf_eval(y_test , preds)

#### Catboosting

In [ ]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(iterations=15)

cat.fit(X_train, y_train)
y_hat = cat.predict(X_test)
cat_matrix = metrics.confusion_matrix(y_test, y_hat)
print(cat_matrix)
cat_report = metrics.classification_report(y_test, y_hat)
print(cat_report)

In [ ]:
pip install catboost

In [ ]:
from itertools import combinations
a = ['ChronicDiseases',"GraduateOrNot",  'FrequentFlyer','Employment Type', 'EverTravelledAbroad' ]
for i in range(len(a)):
    print(list(combinations(a,i)), end = ' ')

In [30]:
import pandas as pd
from numpy import mean, std
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from imblearn.under_sampling import TomekLinks
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from imblearn.under_sampling import *
df = pd.read_csv('./TravelInsurancePrediction.csv')
df['ChronicDiseases'] = df['ChronicDiseases'].map({1: "Yes", 0: 'No'})
df['FrequentFlyer'] = df['FrequentFlyer'].map({'Yes': 1, 'No': 0})
df['EverTravelledAbroad'] = df['EverTravelledAbroad'].map({'Yes': 1, 'No': 0})
df["Employment Type"] = df["Employment Type"].map({"Government Sector" : 1, "Private Sector/Self Employed" : 0})
df['GraduateOrNot'] = df['GraduateOrNot'].map({'Yes': 1, 'No': 0})

ndf = pd.get_dummies(df[['Age', 'AnnualIncome', 'GraduateOrNot','ChronicDiseases',
          'FrequentFlyer', 'FamilyMembers', 'Employment Type',
          'EverTravelledAbroad']])

ndf['FrequentFlyer'] = df['FrequentFlyer']
ndf['TravelInsurance'] = df['TravelInsurance']
X = ndf[list(ndf.columns)[:-1]]
y = ndf[list(ndf.columns)[-1]]
# y = y.astype({'TravelInsurance': int})
# X = preprocessing.StandardScaler().fit(X).transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=10)

In [188]:
df = pd.read_csv('./TravelInsurancePrediction.csv')
df['ChronicDiseases'] = df['ChronicDiseases'].map({1: "Yes", 0: 'No'})
df['FrequentFlyer'] = df['FrequentFlyer'].map({'Yes': 1, 'No': 0})
# df['EverTravelledAbroad'] = df['EverTravelledAbroad'].map({'Yes': 1, 'No': 0})
# df["Employment Type"] = df["Employment Type"].map({"Government Sector" : 1, "Private Sector/Self Employed" : 0})
# df['GraduateOrNot'] = df['GraduateOrNot'].map({'Yes': 1, 'No': 0})
# 
ndf = pd.get_dummies(df[['ChronicDiseases', 'GraduateOrNot','Employment Type','EverTravelledAbroad']])

ndf1 = df[['Age', 'AnnualIncome', 'FrequentFlyer',
           'FamilyMembers',
           'TravelInsurance']]
ndf = pd.concat([ndf, ndf1], axis = 1)
# # ndf['TravelInsurance'] = df['TravelInsurance']
X = ndf[list(ndf.columns)[:-1]]
y = ndf[list(ndf.columns)[-1]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=10)

In [189]:
ndf.head()

,ChronicDiseases_No,ChronicDiseases_Yes,GraduateOrNot_No,GraduateOrNot_Yes,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,EverTravelledAbroad_No,EverTravelledAbroad_Yes,Age,AnnualIncome,FrequentFlyer,FamilyMembers,TravelInsurance
0,0,1,0,1,1,0,1,0,31,400000,0,6,0
1,1,0,0,1,0,1,1,0,31,1250000,0,7,0
2,0,1,0,1,0,1,1,0,34,500000,0,4,1
3,0,1,0,1,0,1,1,0,28,700000,0,3,0
4,0,1,0,1,0,1,1,0,28,700000,1,8,0


In [180]:
X_train.head()

,ChronicDiseases,GraduateOrNot,EverTravelledAbroad,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,Age,AnnualIncome,FrequentFlyer,FamilyMembers
1857,0,1,0,1,0,27,900000,0,4
685,1,1,1,0,1,34,1500000,1,4
1970,0,1,0,0,1,29,500000,0,4
1885,0,1,0,1,0,35,1100000,0,3
895,0,1,1,0,1,28,600000,0,5


In [191]:
# 가장 높은 점수 결정 나무, depth 5
from imblearn.over_sampling import SMOTENC
smote_nc = SMOTENC(categorical_features=[0, 2], random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train)
print(X_resampled.shape)

(2040, 12)


In [193]:
# 가공전 , 6/57
from sklearn import tree
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)

# tree_model.fit(X_resampled, y_resampled)
tree_model.fit(X_train, y_train)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

[[249   8]
 [ 59  82]]
              precision    recall  f1-score   support

           0       0.81      0.97      0.88       257
           1       0.91      0.58      0.71       141

    accuracy                           0.83       398
   macro avg       0.86      0.78      0.80       398
weighted avg       0.84      0.83      0.82       398



In [170]:
from lightgbm import LGBMClassifier
lgbm_wrapper = LGBMClassifier(n_estimators=400)

# LightGBM도 XGBoost와 동일하게 조기 중단 수행 가능. 
evals = [(X_test, y_test)]
lgbm_wrapper.fit(X_resampled, y_resampled, early_stopping_rounds=100, eval_metric="logloss", 
                 eval_set=evals, verbose=True)
y_hat = lgbm_wrapper.predict(X_test)
lgbm_wrapper_matrix = metrics.confusion_matrix(y_test, y_hat)
print(lgbm_wrapper_matrix)
lgbm_wrapper_report = metrics.classification_report(y_test, y_hat)
print(lgbm_wrapper_report)

[1]	valid_0's binary_logloss: 0.655608
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.626449
[3]	valid_0's binary_logloss: 0.602122
[4]	valid_0's binary_logloss: 0.583049
[5]	valid_0's binary_logloss: 0.566604
[6]	valid_0's binary_logloss: 0.553558
[7]	valid_0's binary_logloss: 0.542497
[8]	valid_0's binary_logloss: 0.532616
[9]	valid_0's binary_logloss: 0.524257
[10]	valid_0's binary_logloss: 0.517932
[11]	valid_0's binary_logloss: 0.51247
[12]	valid_0's binary_logloss: 0.508256
[13]	valid_0's binary_logloss: 0.502065
[14]	valid_0's binary_logloss: 0.497786
[15]	valid_0's binary_logloss: 0.495944
[16]	valid_0's binary_logloss: 0.493143
[17]	valid_0's binary_logloss: 0.492166
[18]	valid_0's binary_logloss: 0.490142
[19]	valid_0's binary_logloss: 0.490124
[20]	valid_0's binary_logloss: 0.488984
[21]	valid_0's binary_logloss: 0.48892
[22]	valid_0's binary_logloss: 0.48871
[23]	valid_0's binary_logloss: 0.488385
[24]	valid_0's binary_logloss:

In [171]:
#랜덤 포레스트는 오버샘플링이 좋음
# 오버 언더 조합도 낫베드( 데이터 개수가 늘어날 수록 좋음)
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_resampled , y_resampled)
# rf_clf.fit(X_train , y_train)
y_hat = rf_clf.predict(X_test)
rf_clf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_clf_matrix)
rf_clf_report = metrics.classification_report(y_test, y_hat)
print(rf_clf_report)

[[217  40]
 [ 50  91]]
              precision    recall  f1-score   support

           0       0.81      0.84      0.83       257
           1       0.69      0.65      0.67       141

    accuracy                           0.77       398
   macro avg       0.75      0.74      0.75       398
weighted avg       0.77      0.77      0.77       398

